In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import glob
import re
from openpyxl import load_workbook
from google.colab import drive
from collections import defaultdict

drive.mount('/content/drive')

warnings.filterwarnings('ignore')
path = "/content/drive/MyDrive/GTNP_Borehole_Dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# reformat the file names to end with just ".csv" not ".timeserie.csv"
dfs = []

for filename in glob.glob(os.path.join(path, '*.timeserie.csv')):
  fixed_filename = filename.replace(".timeserie.csv", ".csv")
  os.rename(filename, fixed_filename)

for filename in glob.glob(os.path.join(path, '*.csv')):
  df = pd.read_csv(filename, index_col=None, header=0)
  #print(filename)
  #if df.columns[0]=="Date/Depth":
    #print(filename)
  dfs.append(df)

In [3]:
# Function to process borehole files
def process_borehole_files(directory_path):
    # List all files in the directory, excluding "AATSR" datasets and "Monthly" datasets
    files = [
        os.path.join(directory_path, file) for file in os.listdir(directory_path)
        if file.endswith('.csv') and "AATSR" not in file and "Monthly" not in file
    ]

    # Group files by borehole site name (based on file naming convention)
    site_data = {}
    for file in files:
        # Extract site name and dataset type from the file name using regex
        file_name = os.path.basename(file)

        # Extract site name (e.g., Deadhorse_1)
        site_match = re.search(r"Borehole_\d+-(.*?)-Dataset", file_name)
        site_name = site_match.group(1).strip() if site_match else "Unknown Site"

        # Identify dataset type and data source for proper sheet naming
        if re.search(r"Air[_ ]Temperature", file_name, re.IGNORECASE):
            sheet_name = "Air Temps (Satellite)" if "satellite" in file_name else "Air Temps (Thermistor)"
        elif re.search(r"Ground[_ ]Temperature", file_name, re.IGNORECASE):
            sheet_name = "Ground Temps (Satellite)" if "satellite" in file_name else "Ground Temps (Thermistor)"
        elif re.search(r"Surface[_ ]Temperature", file_name, re.IGNORECASE):
            sheet_name = "Surface Temps (Satellite)" if "satellite" in file_name else "Surface Temps (Thermistor)"
        elif re.search(r"Soil[_ ]Moisture", file_name, re.IGNORECASE):
            sheet_name = "Soil Moisture"
        else:
            sheet_name = "Other Data"

        # Read the CSV file into a DataFrame
        try:
            df = pd.read_csv(file)
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue

        # Collect all sheets for the site
        if site_name not in site_data:
            site_data[site_name] = {}
        site_data[site_name][sheet_name] = df

    # Resolve conflicts for Ground and Surface temperatures
    for site_name, datasets in site_data.items():
        # Prioritize thermistor data over satellite data for Ground Temps
        if "Ground Temps (Thermistor)" in datasets:
            datasets.pop("Ground Temps (Satellite)", None)
        # Prioritize thermistor data over satellite data for Surface Temps
        if "Surface Temps (Thermistor)" in datasets:
            datasets.pop("Surface Temps (Satellite)", None)

    processed_file_paths = []

    # Create an Excel workbook for each site
    for site_name, datasets in site_data.items():
        # Define the output Excel file name
        excel_file_name = f"{site_name.replace(' ', '_')}.xlsx"
        excel_file_path = os.path.join(directory_path, excel_file_name)

        # Write datasets to separate sheets in the Excel workbook
        with pd.ExcelWriter(excel_file_path, engine="openpyxl") as writer:
            for sheet_name, df in datasets.items():
                # Limit sheet names to 31 characters (Excel limitation)
                sheet_name = sheet_name[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        processed_file_paths.append(excel_file_path)
        print(f"Excel file created: {excel_file_path}")
    return processed_file_paths

# Run the function
processed_file_paths = process_borehole_files(path)
print([path for path in processed_file_paths])


Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Deadhorse_2__new_instrumentation_.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Happy_Valley_1_ib.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Franklin_Bluffs__surface_.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Sagwon_MNT.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Happy_Valley_1_b.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Howe_Island_1_b.xlsx
Excel file created: /content/drive/MyDrive/GTNP_Borehole_Dataset/Deadhorse_1__surface_.xlsx
['/content/drive/MyDrive/GTNP_Borehole_Dataset/Deadhorse_2__new_instrumentation_.xlsx', '/content/drive/MyDrive/GTNP_Borehole_Dataset/Happy_Valley_1_ib.xlsx', '/content/drive/MyDrive/GTNP_Borehole_Dataset/Franklin_Bluffs__surface_.xlsx', '/content/drive/MyDrive/GTNP_Borehole_Dataset/Sagwon_MNT.xlsx', '/content/drive/MyDrive/GTNP_Borehole_Dataset/Happy_Valley_1_b.x

In [4]:
def interpolate_ground_temps(excel_file):
    # Open the Excel file
    xls = pd.ExcelFile(excel_file)

    print("Available sheets:", xls.sheet_names)
    # Find Groundtemps
    ground_temp_sheet = next((sheet for sheet in xls.sheet_names if "Ground Temps" in sheet), None)

    if not ground_temp_sheet:
        print("No ground temperature sheet found in the file.")
        return None


    # Load sheet
    df = pd.read_excel(xls, sheet_name=ground_temp_sheet, header=None)

    # Use the first row as the depth index and the first column as timestamps
    depths = df.iloc[0, 1:].values  # Extract depths from the first row, starting at column 2
    timestamps = df.iloc[1:, 0].values  # Extract timestamps from the first column, starting at row 2

    # Extract temperature data starting from the second row and column
    temp_data = df.iloc[1:, 1:].values  # Extract data starting from 2B

    # Create a new DataFrame with depths as column headers and timestamps as row index
    cleaned_df = pd.DataFrame(temp_data, index=timestamps, columns=depths)

    # Replace placeholder -999 with NaN for interpolation
    cleaned_df.replace(-999, np.nan, inplace=True)

    # Drop rows where all temperature values are NaN
    cleaned_df = cleaned_df.dropna(how='all', axis=0)

    # Interpolate using the depth values (accounting for inconsistent intervals)
    for idx, row in cleaned_df.iterrows():
        # Get the valid (non-NaN) temperature values and corresponding depths
        valid_mask = row.notna()
        x_known = cleaned_df.columns[valid_mask]  # Known depths
        y_known = row[valid_mask].values         # Known temperatures

        # Interpolate for all depths
        x_all = cleaned_df.columns  # All depths
        row[:] = np.interp(x_all, x_known, y_known, left=np.nan, right=np.nan)

    # Drop columns that still have any NaN values
    cleaned_df = cleaned_df.dropna(axis=1, how='any')

    # Check if any values are negative
    if (cleaned_df < 0).any().any():
        print("Negative values detected. Converting to Kelvin...")
        cleaned_df += 273.15  # Convert Celsius to Kelvin

    # Save the cleaned sheet into the original file as a new sheet
    with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl') as writer:
        cleaned_df.to_excel(writer, sheet_name="Ground Temps (Cleaned)")

    return cleaned_df



In [5]:
def process_temperature_or_moisture_sheet(excel_file, sheet_type):
    """
    Generic function to process temperature or soil moisture sheets.

    Args:
        excel_file (str): Path to the Excel file.
        sheet_type (str): Type of sheet ("Surface Temps", "Air Temps", or "Soil Moisture").

    Returns:
        pd.DataFrame: Cleaned DataFrame with processed data.
    """
    # Open the Excel file
    xls = pd.ExcelFile(excel_file)

    print("Available sheets:", xls.sheet_names)
    # Find the desired sheet
    sheet = next((sheet for sheet in xls.sheet_names if re.search(sheet_type, sheet, re.IGNORECASE)), None)

    if not sheet:
        print(f"No {sheet_type.lower()} sheet found in the file.")
        return None

    print(f"Processing {sheet_type.lower()} sheet for: {excel_file[45:]}")

    # Load sheet
    df = pd.read_excel(xls, sheet_name=sheet)

    if "moist" in sheet_type.lower():
        # Specific handling for Soil Moisture sheets
        moisture_col = next((col for col in df.columns if re.search(r"moist", str(col), re.IGNORECASE)), None)
        if not moisture_col:
            print("No soil moisture column found. Exiting.")
            return None

        # Keep only the date and soil moisture column
        cleaned_df = df.iloc[:, [0]].copy()  # Select the first column
        cleaned_df[moisture_col] = df[moisture_col]  # Add the soil moisture column


        # Replace -999 with NaN
        cleaned_df[moisture_col].replace(-999, np.nan, inplace=True)

        # Interpolate missing values
        cleaned_df[moisture_col] = cleaned_df[moisture_col].interpolate()

        # Drop rows with remaining NaNs (edges)
        cleaned_df.dropna(inplace=True)

        if cleaned_df[moisture_col].max() > 1:
            cleaned_df[moisture_col] /= 100  # Convert percentages to fractions
        else:
            print("Soil moisture values are not in the range 0-100. No conversion applied.")

        # Save cleaned sheet
        cleaned_sheet_name = "Soil Moisture (Cleaned)"
    else:
        # Handling for temperature sheets

        # Use the first row as column headers and the first column as timestamps
        data_categories = df.iloc[0, 1:].values  # Extract column headers from the first row
        timestamps = df.iloc[1:, 0].values  # Extract timestamps from the first column

        # Extract data starting from the second row and column
        temp_data = df.iloc[1:, 1:].values  # Extract numeric data

        # Create a new DataFrame with proper headers and index
        cleaned_df = pd.DataFrame(temp_data, index=timestamps, columns=data_categories)
        if "index" in cleaned_df.columns:
          cleaned_df.rename(columns={"index": "Date"}, inplace=True)
        # Replace placeholder -999 with NaN
        cleaned_df.replace(-999, np.nan, inplace=True)

        # Check for numeric column headers
        numeric_columns = [col for col in cleaned_df.columns if isinstance(col, (int, float, np.number))]

        if len(numeric_columns) >= 2:
            # Merge numeric columns (assumed temperature columns)
            merged_col_name = f"{sheet_type.split()[0]} Temp [K]"
            cleaned_df[merged_col_name] = cleaned_df[numeric_columns[0]].combine_first(
                cleaned_df[numeric_columns[1]]
            )
            # Drop original numeric columns
            cleaned_df.drop(columns=numeric_columns[:2], inplace=True)
        else:
            # Use regex to find temperature and number-related columns
            temp_col = next((col for col in cleaned_df.columns if re.search(r"temp", str(col), re.IGNORECASE)), None)
            number_col = next((col for col in cleaned_df.columns if re.search(r"number", str(col), re.IGNORECASE)), None)

            if temp_col and number_col:
                cleaned_df.drop(columns=[number_col], inplace=True)
            if temp_col:
                cleaned_df[temp_col] = cleaned_df[temp_col].interpolate()
            else:
                print("No directly labeled column found, proceeding with the first.")
                potential_temp_col = cleaned_df.columns[0]  # Assuming single-column cases
                if cleaned_df[potential_temp_col].isnull().any():
                    print(f"Interpolating missing values in column: {potential_temp_col}")
                    cleaned_df[potential_temp_col] = cleaned_df[potential_temp_col].interpolate()

                # Rename column for consistency
                cleaned_df.rename(columns={potential_temp_col: f"{sheet_type.split()[0]} Temp [K]"}, inplace=True)

        # Drop rows with all NaNs after interpolating
        cleaned_df.dropna(how="all", inplace=True)

        # Convert negative values to Kelvin
        if (cleaned_df < 0).any().any():
            cleaned_df += 273.15  # Convert Celsius to Kelvin

        # Save cleaned sheet
        cleaned_df.reset_index(inplace=True)  # Reset the index to include timestamps as a column
        cleaned_sheet_name = f"{sheet_type.split()[0]} Temps (Cleaned)"

    # Save the processed data back into the Excel file
    with pd.ExcelWriter(excel_file, mode='a', engine='openpyxl') as writer:
        cleaned_df.to_excel(writer, sheet_name=cleaned_sheet_name, index=False)
        print(f"Processed data saved to sheet: {cleaned_sheet_name}")

    return cleaned_df


def consolidate_surface_temps(excel_file):
    """Consolidate surface temperature data."""
    return process_temperature_or_moisture_sheet(excel_file, sheet_type="Surface Temps")


def consolidate_air_temps(excel_file):
    """Consolidate air temperature data."""
    return process_temperature_or_moisture_sheet(excel_file, sheet_type="Air Temps")


def consolidate_soil_moisture(excel_file):
    """Consolidate soil moisture data."""
    return process_temperature_or_moisture_sheet(excel_file, sheet_type="Soil Moisture")

In [6]:
def flatten_ground_temp(excel_file, sheet_name="Ground Temps (Cleaned)"):
    """
    Flattens a 2D ground temperature sheet (date_time × depth) into a 1D vector
    with columns: date_time, depth, and temp. Saves the result as a new sheet.

    Args:
        excel_file (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to process.
    """

    # Load the sheet into a DataFrame
    xls = pd.ExcelFile(excel_file)
    if sheet_name not in xls.sheet_names:
        print(f"Sheet '{sheet_name}' not found in the file.")
        return None

    df = pd.read_excel(xls, sheet_name=sheet_name)

    # Ensure the first column is the date-time
    df.rename(columns={df.columns[0]: "Date"}, inplace=True)

    # Melt the DataFrame to long format
    flattened_df = df.melt(id_vars=["Date"], var_name="Depth", value_name="Temperature")

    # Convert 'Depth' to numeric (if applicable)
    flattened_df["Depth"] = pd.to_numeric(flattened_df["Depth"], errors="coerce")

    # Drop rows with NaN values (e.g., invalid temperatures or depths)
    flattened_df.dropna(inplace=True)

    # Reset the index for a clean DataFrame
    flattened_df.reset_index(drop=True, inplace=True)

    # Save the flattened DataFrame as a new sheet
    with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl") as writer:
        flattened_df.to_excel(writer, sheet_name="Ground Temps (Flattened)", index=False)

    return flattened_df

def process_monthly_avg_ground_temp(excel_file, sheet_name="Ground Temps (Cleaned)"):
    """
    Flattens a 2D ground temperature sheet (date_time × depth) into a 1D vector
    with columns: date_time, depth, and temp. Additionally, calculates the
    monthly average for each depth (distinctly for each year and month), converts
    Celsius to Kelvin (if needed), and saves the result as a new sheet.

    Args:
        excel_file (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to process.
    """
    # Load the sheet into a DataFrame
    xls = pd.ExcelFile(excel_file)
    if sheet_name not in xls.sheet_names:
        print(f"Sheet '{sheet_name}' not found in the file.")
        return None

    df = pd.read_excel(xls, sheet_name=sheet_name)

    # Ensure the first column is the date-time
    df.rename(columns={df.columns[0]: "Date"}, inplace=True)

    # Convert 'Date' to datetime format
    df["Date"] = pd.to_datetime(df["Date"])

    # Melt the DataFrame to long format (flattening)
    flattened_df = df.melt(id_vars=["Date"], var_name="Depth", value_name="Temperature")

    # Drop rows with NaN values
    flattened_df.dropna(inplace=True)

    # Convert 'Depth' to numeric (if applicable)
    flattened_df["Depth"] = pd.to_numeric(flattened_df["Depth"], errors="coerce")
    flattened_df.dropna(inplace=True)  # Remove rows where Depth is NaN after conversion

    # Create 'Year' and 'Month' columns for grouping
    flattened_df["Year"] = flattened_df["Date"].dt.year
    flattened_df["Month"] = flattened_df["Date"].dt.month

    # Group by 'Year', 'Month', and 'Depth' to calculate the average temperature
    monthly_avg_df = (
        flattened_df.groupby(["Year", "Month", "Depth"], as_index=False)["Temperature"]
        .mean()
        .rename(columns={"Temperature": "Avg Temp"})
    )

    # Add a 'Date' column in "YYYY-MM-01" format
    monthly_avg_df["Date"] = pd.to_datetime(
        monthly_avg_df[["Year", "Month"]].assign(DAY=1)
    )

    # Keep only the desired columns: Date, Depth, Avg Temp
    monthly_avg_df = monthly_avg_df[["Date", "Depth", "Avg Temp"]]

    # Save the flattened DataFrame and monthly averages as new sheets
    with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl") as writer:
        # Save the monthly average data
        monthly_avg_df.to_excel(writer, sheet_name="Monthly Average Ground Temp (Flattened)", index=False)

    return monthly_avg_df


In [7]:
def process_monthly_avg_temp(excel_file, sheet_name, output_sheet_name):
    """
    Processes a temperature or moisture sheet (e.g., Air Temps, Surface Temps, Soil Moisture)
    to calculate monthly averages and saves the result as a new sheet.

    Args:
        excel_file (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to process.
        output_sheet_name (str): Name of the output sheet to save monthly averages.

    Returns:
        pd.DataFrame: The monthly average DataFrame.
    """
    # Load the sheet into a DataFrame
    xls = pd.ExcelFile(excel_file)
    if sheet_name not in xls.sheet_names:
        print(f"Sheet '{sheet_name}' not found in the file.")
        return None

    df = pd.read_excel(xls, sheet_name=sheet_name)

    # Ensure the first column is the date-time
    df.rename(columns={df.columns[0]: "Date"}, inplace=True)

    # Convert 'Date' to datetime format
    df["Date"] = pd.to_datetime(df["Date"])

    # Extract year and month for grouping
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month

    # Determine the feature being processed
    feature_column = df.columns[1]
    is_moisture = re.search(r"moisture", feature_column, re.IGNORECASE)

    # Process monthly averages
    if is_moisture:
        # For soil moisture, calculate the monthly average and rename the column
        monthly_avg_df = (
            df.groupby(["Year", "Month"], as_index=False)[feature_column]
            .mean()
            .rename(columns={feature_column: "Avg Moisture"})
        )
        # Add a 'Date' column in "YYYY-MM-01" format
        monthly_avg_df["Date"] = pd.to_datetime(
            monthly_avg_df[["Year", "Month"]].assign(DAY=1)
        )
        # Keep only relevant columns: Date, Avg Moisture
        monthly_avg_df = monthly_avg_df[["Date", "Avg Moisture"]]
    else:
        # For temperatures, calculate the monthly average and rename the column
        monthly_avg_df = (
            df.groupby(["Year", "Month"], as_index=False)[feature_column]
            .mean()
            .rename(columns={feature_column: "Avg Temp"})
        )
        # Add a 'Date' column in "YYYY-MM-01" format
        monthly_avg_df["Date"] = pd.to_datetime(
            monthly_avg_df[["Year", "Month"]].assign(DAY=1)
        )
        # Keep only relevant columns: Date, Avg Temp
        monthly_avg_df = monthly_avg_df[["Date", "Avg Temp"]]

    # Check for negative temperatures and convert to Kelvin if needed
    if not is_moisture and (monthly_avg_df["Avg Temp"] < 0).any():
        print(f"Negative temperatures found in {sheet_name}. Converting to Kelvin...")
        monthly_avg_df["Avg Temp"] += 273.15

    # Save the result as a new sheet
    with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl") as writer:
        monthly_avg_df.to_excel(writer, sheet_name=output_sheet_name, index=False)
        print(f"Saved monthly averages to sheet: {output_sheet_name}")

    return monthly_avg_df


In [8]:
def retain_only_cleaned_sheets(excel_file):
    """
    Retains only sheets with "cleaned" in their names (case-insensitive) in the Excel file.
    Deletes all other sheets.

    Args:
        excel_file (str): Path to the Excel file.
    """
    # Load the workbook
    workbook = load_workbook(excel_file)
    all_sheets = workbook.sheetnames

    # Find sheets to retain (those containing "cleaned")
    cleaned_sheets = [sheet for sheet in all_sheets if "cleaned" in sheet.lower() or "flattened" in sheet.lower()]

    # Delete sheets not in the cleaned_sheets list
    for sheet in all_sheets:
        if sheet not in cleaned_sheets:
            print(f"Deleting sheet: {sheet}")
            del workbook[sheet]

    # Save the updated workbook
    workbook.save(excel_file)
    print(f"Retained only cleaned sheets: {cleaned_sheets}")


In [9]:
total_files = len(processed_file_paths)
for file_to_clean in processed_file_paths:
  _ = interpolate_ground_temps(file_to_clean)
  _ = consolidate_surface_temps(file_to_clean)
  _ = consolidate_air_temps(file_to_clean)
  _ = consolidate_soil_moisture(file_to_clean)
  _ = flatten_ground_temp(file_to_clean)

  _ = process_monthly_avg_ground_temp(file_to_clean)
  _ = process_monthly_avg_temp(
      file_to_clean, sheet_name="Air Temps (Cleaned)", output_sheet_name="Monthly Average Air Temp (Flattened)")
  _ = process_monthly_avg_temp(
      file_to_clean, sheet_name="Surface Temps (Cleaned)", output_sheet_name="Monthly Average Surface Temp (Flattened)")
  _ = process_monthly_avg_temp(
      file_to_clean, sheet_name="Soil Moisture (Cleaned)", output_sheet_name="Monthly Average Soil Moisture (Flattened)")
  retain_only_cleaned_sheets(file_to_clean)

Available sheets: ['Surface Temps (Thermistor)', 'Air Temps (Thermistor)', 'Soil Moisture', 'Ground Temps (Thermistor)']
Negative values detected. Converting to Kelvin...
Available sheets: ['Surface Temps (Thermistor)', 'Air Temps (Thermistor)', 'Soil Moisture', 'Ground Temps (Thermistor)', 'Ground Temps (Cleaned)']
Processing surface temps sheet for: Deadhorse_2__new_instrumentation_.xlsx
Processed data saved to sheet: Surface Temps (Cleaned)
Available sheets: ['Surface Temps (Thermistor)', 'Air Temps (Thermistor)', 'Soil Moisture', 'Ground Temps (Thermistor)', 'Ground Temps (Cleaned)', 'Surface Temps (Cleaned)']
Processing air temps sheet for: Deadhorse_2__new_instrumentation_.xlsx
No directly labeled column found, proceeding with the first.
Processed data saved to sheet: Air Temps (Cleaned)
Available sheets: ['Surface Temps (Thermistor)', 'Air Temps (Thermistor)', 'Soil Moisture', 'Ground Temps (Thermistor)', 'Ground Temps (Cleaned)', 'Surface Temps (Cleaned)', 'Air Temps (Cleaned)'

In [11]:
def consolidate_monthly_averages(processed_file_paths, output_file):
    """
    Consolidates monthly average ground, air, surface temperature,
    and soil moisture across multiple files.

    Args:
        processed_file_paths (list): List of file paths containing processed sheets.
        output_file (str): Output Excel file to save the consolidated data.
    """
    consolidated_data = {
        "Monthly Average Ground Temp": [],
        "Monthly Average Air Temp": [],
        "Monthly Average Surface Temp": [],
        "Monthly Average Soil Moisture": [],
    }

    # Iterate over each processed file
    for file_path in processed_file_paths:
        xls = pd.ExcelFile(file_path)

        # Consolidate ground temperatures
        if "Monthly Average Ground Temp (Flattened)" in xls.sheet_names:
            ground_df = pd.read_excel(xls, sheet_name="Monthly Average Ground Temp (Flattened)")
            ground_df.rename(columns={"Avg Temp": "Avg Ground Temp"}, inplace=True)
            consolidated_data["Monthly Average Ground Temp"].append(ground_df)

        # Consolidate air temperatures
        if "Monthly Average Air Temp (Flattened)" in xls.sheet_names:
            air_df = pd.read_excel(xls, sheet_name="Monthly Average Air Temp (Flattened)")
            air_df.rename(columns={"Avg Temp": "Avg Air Temp"}, inplace=True)
            consolidated_data["Monthly Average Air Temp"].append(air_df)

        # Consolidate surface temperatures
        if "Monthly Average Surface Temp (Flattened)" in xls.sheet_names:
            surface_df = pd.read_excel(xls, sheet_name="Monthly Average Surface Temp (Flattened)")
            surface_df.rename(columns={"Avg Temp": "Avg Surface Temp"}, inplace=True)
            consolidated_data["Monthly Average Surface Temp"].append(surface_df)

        # Consolidate soil moisture
        if "Monthly Average Soil Moisture (Flattened)" in xls.sheet_names:
            moisture_df = pd.read_excel(xls, sheet_name="Monthly Average Soil Moisture (Flattened)")
            moisture_df.rename(columns={"Avg Moisture": "Avg Soil Moisture"}, inplace=True)
            consolidated_data["Monthly Average Soil Moisture"].append(moisture_df)

    # Combine data across all files and resolve conflicts
    consolidated_results = {}
    for key, data_list in consolidated_data.items():
        if data_list:
            combined_df = pd.concat(data_list, ignore_index=True)
            if key == "Monthly Average Ground Temp":
                # For ground temp, consider depth in conflict resolution
                consolidated_results[key] = (
                    combined_df.groupby(["Date", "Depth"], as_index=False)["Avg Ground Temp"].mean()
                )
            elif key == "Monthly Average Soil Moisture":
                # For soil moisture, group only by date
                consolidated_results[key] = (
                    combined_df.groupby("Date", as_index=False)["Avg Soil Moisture"].mean()
                )
            elif key == "Monthly Average Air Temp":
                # For air temps, group only by date
                consolidated_results[key] = (
                    combined_df.groupby("Date", as_index=False)["Avg Air Temp"].mean()
                )
            elif key == "Monthly Average Surface Temp":
                # For surface temps, group only by date
                consolidated_results[key] = (
                    combined_df.groupby("Date", as_index=False)["Avg Surface Temp"].mean()
                )

    # Save consolidated results into a single Excel file
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for sheet_name, df in consolidated_results.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Consolidated data saved to {output_file}")

# Example usage
consolidated_file_path = path + "/consolidated_temperatures_flattened.xlsx"
consolidate_monthly_averages(processed_file_paths, consolidated_file_path)


Consolidated data saved to /content/drive/MyDrive/GTNP_Borehole_Dataset/consolidated_temperatures_flattened.xlsx


In [14]:
def create_master_dataset(consolidated_file_path, output_file_path):
    """
    Combines ground temperature, air temperature, surface temperature,
    and soil moisture into a master dataset and saves it.
    Also saves a version before dropping rows with NaN values.

    Args:
        consolidated_file_path (str): Path to the consolidated Excel file.
        output_file_path (str): Path to save the master dataset as .xlsx and .csv.
    """

    # Load the consolidated Excel file
    xls = pd.ExcelFile(consolidated_file_path)

    # Load each sheet and ensure consistent column names
    ground_temp = pd.read_excel(xls, sheet_name="Monthly Average Ground Temp")
    air_temp = pd.read_excel(xls, sheet_name="Monthly Average Air Temp")
    surface_temp = pd.read_excel(xls, sheet_name="Monthly Average Surface Temp")
    soil_moisture = pd.read_excel(xls, sheet_name="Monthly Average Soil Moisture")

    # Merge datasets based on the "Date" column
    # Preserving Depth column for ground_temp
    master_data = ground_temp.copy()
    master_data = pd.merge(master_data, air_temp.rename(columns={"Avg Air Temp": "Air Temp"}), on="Date", how="left")
    master_data = pd.merge(master_data, surface_temp.rename(columns={"Avg Surface Temp": "Surface Temp"}), on="Date", how="left")
    master_data = pd.merge(master_data, soil_moisture.rename(columns={"Avg Soil Moisture": "Soil Moisture"}), on="Date", how="left")

    # Save the version before dropping rows with NaN
    pre_drop_output_file_path = output_file_path + "_pre_drop"
    pre_drop_sheet_name = "Master Data (Pre-Drop)"
    with pd.ExcelWriter(pre_drop_output_file_path + ".xlsx", engine="openpyxl") as writer:
        master_data.to_excel(writer, sheet_name=pre_drop_sheet_name, index=False)
    master_data.to_csv(pre_drop_output_file_path + ".csv", index=False)

    print(f"Master dataset (pre-drop) saved as {pre_drop_output_file_path}.xlsx and {pre_drop_output_file_path}.csv")

    # Drop rows with any NaN values
    master_data.dropna(inplace=True)

    # Save the cleaned dataset
    cleaned_sheet_name = "Master Consolidated Data"
    with pd.ExcelWriter(output_file_path + ".xlsx", engine="openpyxl") as writer:
        master_data.to_excel(writer, sheet_name=cleaned_sheet_name, index=False)
    master_data.to_csv(output_file_path + ".csv", index=False)

    print(f"Cleaned master dataset saved as {output_file_path}.xlsx and {output_file_path}.csv")

# Example usage
output_file_path = path + "/consolidated_data_master"  # Replace with your output file path
create_master_dataset(consolidated_file_path, output_file_path)


Master dataset (pre-drop) saved as /content/drive/MyDrive/GTNP_Borehole_Dataset/consolidated_data_master_pre_drop.xlsx and /content/drive/MyDrive/GTNP_Borehole_Dataset/consolidated_data_master_pre_drop.csv
Cleaned master dataset saved as /content/drive/MyDrive/GTNP_Borehole_Dataset/consolidated_data_master.xlsx and /content/drive/MyDrive/GTNP_Borehole_Dataset/consolidated_data_master.csv
